In [1]:
import os
import sys
import math
import logging
from pathlib import Path

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from enadepy import loaders

In [3]:
df = loaders.read_raw('../data/raw/microdados_enade_2018.csv', low_memory=False)

In [4]:
df2 = loaders.read_raw('../data/raw/microdados_enade_2017.csv', low_memory=False)

In [5]:
df3 = loaders.read_raw('../data/interim/microdados_enade_2016.csv', low_memory=False)

In [6]:
result = pd.concat([df3.query('CO_MUNIC_CURSO == 3502804'), df2.query('CO_MUNIC_CURSO == 3502804'), df.query('CO_MUNIC_CURSO == 3502804')])

In [7]:
result.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
NU_ANO,2075.0,2.017184e+03,7.679209e-01,2016.0,2017.0,2017.00,2018.0,2018.0
CO_IES,2075.0,2.351158e+03,2.347572e+03,56.0,845.0,1418.00,4522.0,15697.0
CO_CATEGAD,2075.0,6.523145e+03,5.269747e+03,2.0,5.0,10005.00,10007.0,17634.0
CO_ORGACAD,2075.0,1.002198e+04,3.386354e+00,10020.0,10020.0,10020.00,10022.0,10028.0
CO_GRUPO,2075.0,1.510725e+03,2.322988e+03,1.0,5.0,23.00,3501.0,6306.0
CO_CURSO,2075.0,7.461795e+05,1.231741e+06,3161.0,20642.0,96729.00,1147234.0,5000425.0
CO_MODALIDADE,2075.0,1.000000e+00,0.000000e+00,1.0,1.0,1.00,1.0,1.0
CO_MUNIC_CURSO,2075.0,3.502804e+06,0.000000e+00,3502804.0,3502804.0,3502804.00,3502804.0,3502804.0
CO_UF_CURSO,2075.0,3.500000e+01,0.000000e+00,35.0,35.0,35.00,35.0,35.0
CO_REGIAO_CURSO,2075.0,3.000000e+00,0.000000e+00,3.0,3.0,3.00,3.0,3.0


In [8]:
result.to_csv('../data/interim/microdados_enade_aracatuba_2016a2018.csv', index=False)